### Whisp a feature collection

Setup
- NB use a virtual environment to avoid altering your python environment (https://docs.python.org/3/tutorial/venv.html)

In [1]:
# Earth Engine and Common Libraries|
import ee
from pathlib import Path

# Authenticate and initialize Earth Engine. 
try:
    ee.Initialize()  # Try to use existing credentials first
except Exception:
    ee.Authenticate() # Authenticate may open a browser window
    ee.Initialize()

# NB if not working add your cloud project: ee.Initialize(project="your_gee_cloud_project_name")


Successfully saved authorization token.


=== DEBUGGING RISK COLUMNS FOR COUNTRIES: ['NIC'] ===

Converted ISO3 ['NIC'] to ISO2 ['NI']
Original lookup table has 200 rows
Available ISO2 codes: [nan 'CO' 'BR' 'CI']

Filtered lookup table has 172 rows

Indicator 1 - Treecover:
  Theme: treecover
  Columns: ['EUFO_2020', 'GLAD_Primary', 'TMF_undist', 'GFC_TC_2020', 'Forest_FDaP', 'ESA_TC_2020']
  Count: 6

Indicator 2 - Commodities:
  Theme: commodities
  Columns: ['TMF_plant', 'Oil_palm_Descals', 'Oil_palm_FDaP', 'Coffee_FDaP', 'Cocoa_FDaP', 'Cocoa_ETH', 'Rubber_FDaP', 'Rubber_RBGE', 'Soy_Song_2020']
  Count: 9

Indicator 3 - Disturbance Before:
  Theme: disturbance_before
  Columns: ['TMF_deg_before_2020', 'TMF_def_before_2020', 'GFC_loss_before_2020', 'ESA_fire_before_2020', 'MODIS_fire_before_2020', 'RADD_before_2020']
  Count: 6

Indicator 4 - Disturbance After:
  Theme: disturbance_after
  Columns: ['TMF_deg_after_2020', 'TMF_def_after_2020', 'GFC_loss_after_2020', 'MODIS_fire_after_2020', 'RADD_after_2020']
  Count: 5

Indi

,name,order,ISO2_code,theme,theme_timber,use_for_risk,use_for_risk_timber,exclude_from_output,col_type,is_nullable,is_required,corresponding_variable
0,EUFO_2020,10,NaN,treecover,naturally_reg_2020,1,1,0,float32,1,0,g_jrc_gfc_2020_prep
1,GLAD_Primary,20,NaN,treecover,primary,1,1,0,float32,1,0,g_glad_pht_prep
2,TMF_undist,30,NaN,treecover,primary,1,1,0,float32,1,0,g_jrc_tmf_undisturbed_prep
3,GFC_TC_2020,50,NaN,treecover,naturally_reg_2020,1,1,0,float32,1,0,g_glad_gfc_10pc_prep
4,Forest_FDaP,60,NaN,treecover,naturally_reg_2020,1,1,0,float32,1,0,g_glad_gfc_10pc_prep
...,...,...,...,...,...,...,...,...,...,...,...,...
167,Coffee_FDaP_2023,2111,NaN,NaN,agri_after_2020,0,1,0,float32,1,0,g_fdap_coffee_2023_prep
168,Cocoa_2023_FDaP,2120,NaN,NaN,agri_after_2020,0,1,0,float32,1,0,g_fdap_cocoa_2023_prep
169,ESRI_2023_crop,2130,NaN,NaN,agri_after_2020,0,1,0,float32,1,0,g_esri_2023_crop_prep
170,GLC_FCS30D_crop_2022,2140,NaN,NaN,agri_after_2020,0,1,0,float32,1,0,g_glc_fcs30d_crop_2022_prep


In [18]:
# Debug with ISO2 codes
debug_risk_columns_by_country(['co'])  # Nicaragua, Brazil

# # Debug with ISO3 codes (will convert to ISO2)
# debug_risk_columns_by_country(['NIC', 'BRA'], iso_format="ISO3")

# Debug with no country filter (shows all columns)
debug_risk_columns_by_country(None)

=== DEBUGGING RISK COLUMNS FOR COUNTRIES: ['co'] ===

Original lookup table has 200 rows
Available ISO2 codes: [nan 'CO' 'BR' 'CI']

Filtered lookup table has 174 rows

Indicator 1 - Treecover:
  Theme: treecover
  Columns: ['EUFO_2020', 'GLAD_Primary', 'TMF_undist', 'GFC_TC_2020', 'Forest_FDaP', 'ESA_TC_2020', 'nCO_ideam_forest_2020']
  Count: 7

Indicator 2 - Commodities:
  Theme: commodities
  Columns: ['TMF_plant', 'Oil_palm_Descals', 'Oil_palm_FDaP', 'Coffee_FDaP', 'Cocoa_FDaP', 'Cocoa_ETH', 'Rubber_FDaP', 'Rubber_RBGE', 'Soy_Song_2020', 'nCO_ideam_eufo_commission_2020']
  Count: 10

Indicator 3 - Disturbance Before:
  Theme: disturbance_before
  Columns: ['TMF_deg_before_2020', 'TMF_def_before_2020', 'GFC_loss_before_2020', 'ESA_fire_before_2020', 'MODIS_fire_before_2020', 'RADD_before_2020']
  Count: 6

Indicator 4 - Disturbance After:
  Theme: disturbance_after
  Columns: ['TMF_deg_after_2020', 'TMF_def_after_2020', 'GFC_loss_after_2020', 'MODIS_fire_after_2020', 'RADD_after_20

,name,order,ISO2_code,theme,theme_timber,use_for_risk,use_for_risk_timber,exclude_from_output,col_type,is_nullable,is_required,corresponding_variable
0,EUFO_2020,10,NaN,treecover,naturally_reg_2020,1,1,0,float32,1,0,g_jrc_gfc_2020_prep
1,GLAD_Primary,20,NaN,treecover,primary,1,1,0,float32,1,0,g_glad_pht_prep
2,TMF_undist,30,NaN,treecover,primary,1,1,0,float32,1,0,g_jrc_tmf_undisturbed_prep
3,GFC_TC_2020,50,NaN,treecover,naturally_reg_2020,1,1,0,float32,1,0,g_glad_gfc_10pc_prep
4,Forest_FDaP,60,NaN,treecover,naturally_reg_2020,1,1,0,float32,1,0,g_glad_gfc_10pc_prep
...,...,...,...,...,...,...,...,...,...,...,...,...
167,Coffee_FDaP_2023,2111,NaN,NaN,agri_after_2020,0,1,0,float32,1,0,g_fdap_coffee_2023_prep
168,Cocoa_2023_FDaP,2120,NaN,NaN,agri_after_2020,0,1,0,float32,1,0,g_fdap_cocoa_2023_prep
169,ESRI_2023_crop,2130,NaN,NaN,agri_after_2020,0,1,0,float32,1,0,g_esri_2023_crop_prep
170,GLC_FCS30D_crop_2022,2140,NaN,NaN,agri_after_2020,0,1,0,float32,1,0,g_glc_fcs30d_crop_2022_prep


In [19]:
def test_risk_with_countries(df, national_codes):
    """Test risk calculation and verify columns are included"""
    
    # Run risk calculation
    result_df = whisp_risk(
        df=df,
        national_codes=national_codes,
        explicit_unit_type="percent"  # or "ha"
    )
    
    # Check which risk columns were added
    risk_columns = [col for col in result_df.columns if 'risk' in col.lower() or 'ind_' in col.lower()]
    
    print(f"Risk columns added: {risk_columns}")
    
    # Check specific indicator columns
    indicator_columns = [col for col in result_df.columns if col.startswith('Ind_')]
    print(f"Indicator columns: {indicator_columns}")
    
    return result_df


In [ ]:
# Install openforis-whisp (uncomment line if not already installed)
# !pip install --pre openforis-whisp

# NB for editable mode install via your terminal with: pip install -e .[dev]

In [31]:

# Test it
result = test_risk_with_countries(df_formatted_stats, ['CO'])  # ISO2 for Nicaragua

NameError: name 'whisp_risk' is not defined

In [32]:
import openforis_whisp as whisp

Get a feature collection

In [33]:
# GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("geojson_example.geojson")
GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path(r"C:\Users\Arnell\Downloads\predio.geojson")


Whisp it

In [34]:
# Choose countries to process (currently three countries: 'co', 'ci', 'br')
iso2_codes_list = ['co', 'ci', 'br']  # Example ISO2 codes for including country specific data


In [35]:
df_formatted_stats = whisp.whisp_formatted_stats_geojson_to_df(
    input_geojson_filepath=GEOJSON_EXAMPLE_FILEPATH,
    national_codes=iso2_codes_list, # optional - By default national datasets are not included unless specified here.
    # unit_type='percent', # optional - to change unit type. Default is 'ha'. 
)

Reading GeoJSON file from: C:\Users\Arnell\Downloads\predio.geojson
['Area', 'Oil_palm_Descals', 'European_Primary_Forest', 'ESA_fire_before_2020', 'ESA_fire_2001', 'ESA_fire_2002', 'ESA_fire_2003', 'ESA_fire_2004', 'ESA_fire_2005', 'ESA_fire_2006', 'ESA_fire_2007', 'ESA_fire_2008', 'ESA_fire_2009', 'ESA_fire_2010', 'ESA_fire_2011', 'ESA_fire_2012', 'ESA_fire_2013', 'ESA_fire_2014', 'ESA_fire_2015', 'ESA_fire_2016', 'ESA_fire_2017', 'ESA_fire_2018', 'ESA_fire_2019', 'ESA_fire_2020', 'ESA_TC_2020', 'ESRI_2023_crop', 'ESRI_2023_TC', 'Cocoa_ETH', 'Cocoa_2023_FDaP', 'Cocoa_FDaP', 'Coffee_FDaP', 'Coffee_FDaP_2023', 'Forest_FDaP', 'Oil_palm_2023_FDaP', 'Oil_palm_FDaP', 'Rubber_2023_FDaP', 'Rubber_FDaP', 'GFT_naturally_regenerating', 'GFT_planted_plantation', 'GFT_primary', 'GFC_TC_2020', 'GFC_loss_after_2020', 'GFC_loss_before_2020', 'GFC_loss_year_2001', 'GFC_loss_year_2002', 'GFC_loss_year_2003', 'GFC_loss_year_2004', 'GFC_loss_year_2005', 'GFC_loss_year_2006', 'GFC_loss_year_2007', 'GFC_l

In [36]:
df_formatted_stats

,plotId,external_id,Area,Geometry_type,Country,ProducerCountry,Admin_Level_1,Centroid_lon,Centroid_lat,Unit,...,nBR_MapBiomas_col9_palmoil_2020,nBR_MapBiomas_col9_pc_2020,nBR_INPE_TCamz_cer_annual_2020,nBR_MapBiomas_col9_soy_2020,nBR_MapBiomas_col9_annual_crops_2020,nBR_INPE_TCamz_pasture_2020,nBR_INPE_TCcer_pasture_2020,nBR_MapBiomas_col9_pasture_2020,nCI_Cocoa_bnetd,geo
0,1,NaN,1.672,Polygon,COL,CO,Nariño,-78.678524,1.663318,ha,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'type': 'Polygon', 'coordinates': [[[-78.6791..."
1,2,NaN,0.829,Polygon,COL,CO,Nariño,-78.676262,1.667044,ha,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'type': 'Polygon', 'coordinates': [[[-78.6769..."
2,3,NaN,0.225,Polygon,COL,CO,Bogota Capital District,-74.135853,4.686152,ha,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'type': 'Polygon', 'coordinates': [[[-74.1362..."
3,4,NaN,0.580,Polygon,COL,CO,Bogota Capital District,-74.130931,4.688233,ha,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'type': 'Polygon', 'coordinates': [[[-74.1315..."
4,5,NaN,0.010,Point,COL,CO,Nariño,-78.678023,1.658215,ha,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'type': 'Point', 'coordinates': [-78.678023, ..."
5,6,NaN,0.095,Polygon,COL,CO,Nariño,-78.684483,1.679653,ha,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'type': 'Polygon', 'coordinates': [[[-78.6846..."
6,7,NaN,0.010,Point,COL,CO,Nariño,-78.676887,1.667175,ha,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'type': 'Point', 'coordinates': [-78.676887, ..."
7,8,NaN,0.140,Polygon,COL,CO,Nariño,-78.674452,1.665851,ha,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'type': 'Polygon', 'coordinates': [[[-78.6747..."
8,9,NaN,0.193,Polygon,COL,CO,Caldas,-75.479155,5.057414,ha,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'type': 'Polygon', 'coordinates': [[[-75.4795..."


In [37]:
# Define the output folder (if running in Sepal change path to preferred folder) 
# e.g. out_directory = Path.home() / 'module_results/whisp/'
out_directory = Path.home() / 'downloads'

# Define the output file path for CSV
csv_output_file = out_directory / 'whisp_output_table.csv'

# Save the CSV file
df_formatted_stats.to_csv(path_or_buf=csv_output_file, index=False)
print(f"Table with risk columns saved to: {csv_output_file}")

Table with risk columns saved to: C:\Users\Arnell\downloads\whisp_output_table.csv


Calculate risk category

In [27]:
# add risk columns to end of dataframe
df_w_risk = whisp.whisp_risk(df=df_formatted_stats,
                             national_codes=iso2_codes_list # optional - By default national datasets are not included. This should align with the national_codes used above.
            )

Using unit type: ha


Display table with risk columns

In [28]:
df_w_risk

,plotId,external_id,Area,Geometry_type,Country,ProducerCountry,Admin_Level_1,Centroid_lon,Centroid_lat,Unit,...,Ind_05_primary_2020,Ind_06_nat_reg_forest_2020,Ind_07_planted_plantations_2020,Ind_08_planted_plantations_after_2020,Ind_09_treecover_after_2020,Ind_10_agri_after_2020,Ind_11_logging_concession_before_2020,risk_pcrop,risk_acrop,risk_timber
0,1,NaN,1.672,Polygon,COL,CO,Nariño,-78.678524,1.663318,ha,...,yes,yes,no,no,no,yes,no,low,low,low
1,2,NaN,0.829,Polygon,COL,CO,Nariño,-78.676262,1.667044,ha,...,yes,yes,no,no,no,yes,no,low,low,low
2,3,NaN,0.225,Polygon,COL,CO,Bogota Capital District,-74.135853,4.686152,ha,...,no,no,no,no,no,no,no,low,low,high
3,4,NaN,0.580,Polygon,COL,CO,Bogota Capital District,-74.130931,4.688233,ha,...,no,no,no,no,no,no,no,low,low,high
4,5,NaN,0.010,Point,COL,CO,Nariño,-78.678023,1.658215,ha,...,yes,yes,no,no,no,no,no,low,low,low
5,6,NaN,0.095,Polygon,COL,CO,Nariño,-78.684483,1.679653,ha,...,yes,yes,no,no,no,no,no,low,low,low
6,7,NaN,0.010,Point,COL,CO,Nariño,-78.676887,1.667175,ha,...,yes,yes,no,no,no,yes,no,low,low,low
7,8,NaN,0.140,Polygon,COL,CO,Nariño,-78.674452,1.665851,ha,...,yes,yes,no,no,no,yes,no,low,low,low
8,9,NaN,0.193,Polygon,COL,CO,Caldas,-75.479155,5.057414,ha,...,no,yes,no,no,no,no,no,low,more_info_needed,more_info_needed


Export table to CSV

In [29]:
# Define the output folder 
# e.g. in running in Sepal this might be: Path.home() / 'module_results/whisp/'
out_directory = Path.home() / 'downloads'

# Define the output file path for CSV
csv_output_file = out_directory / 'whisp_output_table_w_risk.csv'

# Save the CSV file
df_w_risk.to_csv(path_or_buf=csv_output_file, index=False)
print(f"Table with risk columns saved to: {csv_output_file}")

Table with risk columns saved to: C:\Users\Arnell\downloads\whisp_output_table_w_risk.csv


Export to GeoJSON (optional)

In [30]:
# Define the output file path for GeoJSON
geojson_output_file = out_directory / 'whisp_output_geo_w_risk.geojson'

# Save the GeoJSON file
whisp.convert_df_to_geojson(df_w_risk, geojson_output_file)  # builds a geojson file containing Whisp columns. Uses the geometry column "geo" to create the spatial features.
print(f"GeoJSON file saved to: {geojson_output_file}")

GeoJSON saved to C:\Users\Arnell\downloads\whisp_output_geo_w_risk.geojson
GeoJSON file saved to: C:\Users\Arnell\downloads\whisp_output_geo_w_risk.geojson
